In [1]:
import numpy as np
import pandas as pd
from statsbombpy import sb
import networkx as nx

In [2]:
#Import competition data to find season we are interested in: La Liga - 2020/2021
competition_df = sb.competitions()
season = competition_df[ (competition_df.competition_name == 'La Liga') & (competition_df.season_name == '2020/2021') ]

C:\Users\Lawrence\AppData\Local\Programs\Python\Python39\lib\site-packages\statsbombpy\api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)


In [3]:
#Find the matches where Barcelona played. Could be as home team or away team
season_matches = sb.matches(competition_id = season.competition_id.iloc[0],season_id = season.season_id.iloc[0])
barcelona_matches = season_matches[ (season_matches.home_team == 'Barcelona') | ( season_matches.away_team == 'Barcelona') ]
#Note that matches are not in date order
sb.events(barcelona_matches.match_id[0]).columns

Index(['ball_receipt_outcome', 'ball_recovery_recovery_failure',
       'block_deflection', 'block_offensive', 'carry_end_location',
       'clearance_aerial_won', 'clearance_body_part', 'clearance_head',
       'clearance_left_foot', 'clearance_other', 'clearance_right_foot',
       'counterpress', 'dribble_nutmeg', 'dribble_outcome', 'dribble_overrun',
       'duel_outcome', 'duel_type', 'duration', 'foul_committed_advantage',
       'foul_committed_card', 'foul_committed_penalty', 'foul_won_advantage',
       'foul_won_defensive', 'foul_won_penalty', 'goalkeeper_body_part',
       'goalkeeper_end_location', 'goalkeeper_outcome', 'goalkeeper_position',
       'goalkeeper_technique', 'goalkeeper_type', 'id', 'index',
       'interception_outcome', 'location', 'match_id', 'minute',
       'miscontrol_aerial_won', 'off_camera', 'out', 'pass_aerial_won',
       'pass_angle', 'pass_assisted_shot_id', 'pass_body_part', 'pass_cross',
       'pass_end_location', 'pass_goal_assist', 'pass_hei

In [17]:
shot_minute = pd.DataFrame()

#Iterates through each match and finds the barca shots, records the minute, second and what possesion it was in
for match_id_iter in barcelona_matches.match_id[0:1]:
    match_iter = sb.events(match_id=match_id_iter)
    shots = match_iter[(match_iter.type == 'Shot') & (match_iter.team == 'Barcelona')]
    shot_minute = pd.concat([shot_minute, shots[['minute','second','possession']]], ignore_index=True)
    #Creates new data frames, dont know if this is necessary
    previous_events = pd.DataFrame()
    ten_previous_events = pd.DataFrame()
    ten_previous_events['y_location'] = np.nan
    #Iterates through each shot event and find the events in an entire minute previous, ONLY COUNTS THE EVENTS IF THEY WERE IN THE SAME 'POSSESSION'
    for i,row in shot_minute.iterrows():
        
        previous_events = match_iter[(((match_iter.minute == row.minute) & (match_iter.second <= row.second )) | ((match_iter.minute == (row.minute - 1)) & (match_iter.second >= row.second ))) & (match_iter.possession == row.possession)]
        #Finding length of this list to allow for loop
        pe_length = len(previous_events.index)
        #For loop finds the last x events unless the previous events have less than x entries
        x = 10
        if pe_length< x:
            ten_previous_events = previous_events.iloc[pe_length:-1]
        else:
            ten_previous_events = previous_events.iloc[-x:-1]

        #I dont know where its gone to shit but it has
        # the timestamps are out of order for each set of previous events
 
        
        blah = ten_previous_events.minute
        ble = ten_previous_events.second
        
        for j,row in ten_previous_events.iterrows():
            ten_previous_events.loc[j,'y_location'] = row.location[1]



ten_previous_events.y_location.mean()

22.9